## Import the necessary libraries

In [60]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline

## Set hyper-parameters

In [61]:
noise_dimension = 10
training_epochs = 30
batch_size = 32
learning_rate = 0.001
run_training = True

## Read in the MNIST-Dataset and reset the default tf.graph (just in case ;-) )

In [62]:
mnist = input_data.read_data_sets('./MNIST_data/',one_hot=True)
tf.reset_default_graph()
init = tf.global_variables_initializer()

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


## Define helper-functions for convenient acces, using the tf.layers-API here

In [63]:
def generator(z, reuse=None):
    with tf.variable_scope('gen',reuse=reuse):
        alpha = 0.01
        hidden1 = tf.layers.dense(inputs=z,units=128)
        hidden1 = tf.maximum(hidden1*alpha,hidden1)
        hidden2 = tf.layers.dense(hidden1,units=128)
        hidden2 = tf.maximum(hidden2*alpha,hidden2)
        outputs = tf.layers.dense(hidden2,units=784,activation=tf.nn.tanh)
        return outputs

In [64]:
def discriminator(X, reuse=None):
    with tf.variable_scope('dis',reuse=reuse):
        alpha = 0.01
        hidden1 = tf.layers.dense(inputs=X,units=128)
        hidden1 = tf.maximum(hidden1*alpha,hidden1)
        hidden2 = tf.layers.dense(hidden1,units=128)
        hidden2 = tf.maximum(hidden2*alpha,hidden2)
        logits = tf.layers.dense(hidden2,units=1)
        outputs = tf.sigmoid(logits)
        return outputs, logits

In [65]:
def loss_func(logits_in,real_images=False):
    if real_images:
        labels = tf.ones_like(logits_in)*0.9
    else:
        labels = tf.zeros_like(logits_in)
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_in,labels=labels))

## Generate the necessary tf.placeholders

In [66]:
real_images = tf.placeholder(tf.float32,shape=[None,784])
z = tf.placeholder(tf.float32,shape=[None,noise_dimension])

## Generate the Graph-Objects for the network-ouputs

In [67]:
G = generator(z)
D_output_real, D_logits_real = discriminator(real_images)
D_output_fake, D_logits_fake = discriminator(G,reuse=True)

## Calculate the network losses

In [68]:
D_real_loss = loss_func(D_logits_real,real_images=True)
D_fake_loss = loss_func(D_logits_fake,real_images=False)
D_loss = D_real_loss + D_fake_loss
G_loss = loss_func(D_logits_fake,real_images=True) #Here our goal is to "fool" the discriminator with oposite labels

## Generate the Optimizers

In [69]:
#Use listcomprehensions for quick access to the tunable variables
tvars = tf.trainable_variables()
d_vars = [var for var in tvars if 'dis' in var.name]
g_vars = [var for var in tvars if 'gen' in var.name]
D_trainer = tf.train.AdamOptimizer(learning_rate).minimize(D_loss,var_list=d_vars)
G_trainer = tf.train.AdamOptimizer(learning_rate).minimize(G_loss,var_list=g_vars)

## Start the training-session

In [80]:
if run_training:
    with tf.Session() as sess:
        sess.run(init)
        
        
        for epoch in range(training_epochs):      
            num_batches = mnist.train.num_examples // batch_size
            
            for i in range(num_batches):
                batch = mnist.train.next_batch(batch_size)
                batch_images = batch[0].reshape((batch_size,28*28))
                batch_images = batch_images * 2 - 1 #Transformation for use with tanh at the output of the generator
                batch_z = np.random.uniform(-1,1,size=(batch_size,noise_dimension))
                sess.run([D_trainer,G_trainer],feed_dict={real_images:batch_images,z:batch_z})

            print('Progess epoch {}/{}'.format(epoch,training_epochs))
            
            #sample_z = np.random.uniform(-1,1,size=(1,noise_dimension))
            #gen_sample = sess.run(G,feed_dict={z:sample_z})
            
            #plt.imshow(gen_sample.reshape(28,28))

FailedPreconditionError: Attempting to use uninitialized value gen/dense/kernel
	 [[Node: gen/dense/kernel/read = Identity[T=DT_FLOAT, _class=["loc:@gen/dense/kernel"], _device="/job:localhost/replica:0/task:0/cpu:0"](gen/dense/kernel)]]

Caused by op 'gen/dense/kernel/read', defined at:
  File "C:\Users\MacenkaJ\Desktop\_Aufgeraeumt\PY_Test\WinPython-64bit-3.5.4.1Qt5\python-3.5.4.amd64\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\MacenkaJ\Desktop\_Aufgeraeumt\PY_Test\WinPython-64bit-3.5.4.1Qt5\python-3.5.4.amd64\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\MacenkaJ\Desktop\_Aufgeraeumt\PY_Test\WinPython-64bit-3.5.4.1Qt5\python-3.5.4.amd64\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\MacenkaJ\Desktop\_Aufgeraeumt\PY_Test\WinPython-64bit-3.5.4.1Qt5\python-3.5.4.amd64\lib\site-packages\traitlets\config\application.py", line 658, in launch_ins